In [1]:
from multiprocessing import Pool #並列計算

import pandas as pd
import numpy as np

from scipy import signal
from scipy import fftpack

from tqdm import tqdm_notebook as tqdm
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams

import datetime
import os
import re
import glob
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
path = '../../output/1_波浪/water_level/20210825_all_data/*010-010.csv'
filelist = glob.glob(path)
filelist.sort()

In [3]:
filelist

['../../output/1_波浪/water_level/20210825_all_data/2010-01-16_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-17_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-18_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-19_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-20_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-21_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-22_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-23_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-24_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-25_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-26_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-27_010-010.csv',
 '../../output/1_波浪/water_level/20210825_all_data/2010-01-28_010-010.csv',
 '../../output/1_波浪/water

In [3]:
filelist = pd.Series(filelist).str[-22:]

# データフレーム作る
# 24時間フレーム数は172800
# 必要な20分フレームは2400  1時間が7200
n_frame = 2400# 20分ごと
time_step=0.5
han = signal.hann(n_frame) #ハニング窓を作成
# ham = signal.hamming(n_frame) # ハミング窓を作成

In [5]:
# lowess

In [6]:
def clc_lowess(freq_, power_, frac_no):
    power_lowess = lowess(power_, range(len(freq_)),  frac=1/frac_no)[:,1]
    return 1 / freq_[np.argmax(power_lowess)]

In [7]:
path = '../../output/1_波浪/water_level/20210825_all_data/'

def calc_spectral(filelist_):
   
    # 並列計算のため、filelist_はリストの引数として格納
    df = pd.read_csv(path + filelist_)

    df_freqs = pd.DataFrame()
    df_freq_all = pd.DataFrame()
    
    for j in range(72):
        try:
            df_split = df.iloc[(n_frame * j):(n_frame * (j+1)),:].reset_index(drop=True)

            df_split['datetime'] = pd.to_datetime(df_split['datetime'], format='%Y-%m-%d')
            df_split['date'] = df_split['datetime'].apply(lambda x: str(x.year) + '-' + \
                                                          str(x.month) + '-' + \
                                                          str(x.day) + ' ' + \
                                                          str(x.hour) + ':' + \
                                                          str(x.minute))
            # グラフ名用に日付抽出
            date = df_split['datetime'].apply(lambda x: str(x.year) + '-' + \
                                                          str(x.month) + '-' + \
                                                          str(x.day))[0]

            date_time = str(df_split['date'][0])


            df_split['date'] = pd.to_datetime(df_split['date'])

            #FFT
            sample_freq = fftpack.fftfreq(len(df_split), d=time_step)  # 周波数軸を生成
            # 窓をかける
            y_fft = np.fft.fft((han)*(df_split['water_level']-np.mean(df_split['water_level'])))/(n_frame/2)  # フーリエスペクトル（複素数）
            pidxs = np.where(sample_freq > 0)



            freqs, power = sample_freq[pidxs], np.abs(y_fft)[pidxs]
            mfreq = freqs[freqs >= 10**(-2)][power[freqs >= 10**(-2)].argmax()] # ピーク周波数

            dfs = pd.DataFrame([{'Datetime' : date_time,
                            'Peak_Freq' : mfreq,
                            'Peak_Cyc_T_max' : 1 / mfreq  ,
                            'Peak_Cyc_T_lowess_1p15' : clc_lowess(freqs, power, 15),
                            'Peak_Cyc_T_lowess_1p30' : clc_lowess(freqs, power, 30),
                            'Peak_Cyc_T_lowess_1p40' : clc_lowess(freqs, power, 40),
                            'Peak_Cyc_T_lowess_1p80' : clc_lowess(freqs, power, 80)
                            }])

            df_freqs = df_freqs.append(dfs)

        except:
            with open('log.txt', 'a') as f:
                print(datetime.datetime.now(), '___error file : ', filelist[i], file=f)
                print(datetime.datetime.now(), '___error frame : ', j, file=f)
                continue

    df_freq_all = df_freq_all.append(df_freqs)
    df_freq_all.to_csv("../../output/1_波浪/water_level/20210908_output_fft_all_data_lowess_r1/fft_han_lowess_" +  filelist_)

        
if __name__ == "__main__":
    p = Pool(16)
    p.map( calc_spectral, filelist )

NameError: name 'i' is not defined

In [4]:
# path = '../../../output/1_波浪/water_level/all_data/'

# def calc_spectral(filelist_):
   
    
#     df = pd.read_csv(path + filelist_)

#     df_freqs = pd.DataFrame()
#     df_freq_all = pd.DataFrame()
    
#     for j in range(72):
#         try:
#             df_split = df.iloc[(n_frame * j):(n_frame * (j+1)),:].reset_index(drop=True)

#             df_split['datetime'] = pd.to_datetime(df_split['datetime'], format='%Y-%m-%d')
#             df_split['date'] = df_split['datetime'].apply(lambda x: str(x.year) + '-' + \
#                                                           str(x.month) + '-' + \
#                                                           str(x.day) + ' ' + \
#                                                           str(x.hour) + ':' + \
#                                                           str(x.minute))
#             # グラフ名用に日付抽出
#             date = df_split['datetime'].apply(lambda x: str(x.year) + '-' + \
#                                                           str(x.month) + '-' + \
#                                                           str(x.day))[0]

#             date_time = str(df_split['date'][0])


#             df_split['date'] = pd.to_datetime(df_split['date'])

#             #FFT
#             sample_freq = fftpack.fftfreq(len(df_split), d=time_step)  # 周波数軸を生成
#             # 窓をかける
#             y_fft = np.fft.fft((han)*(df_split['water_level']-np.mean(df_split['water_level'])))/(n_frame/2)  # フーリエスペクトル（複素数）
#             pidxs = np.where(sample_freq > 0)



#             freqs, power = sample_freq[pidxs], np.abs(y_fft)[pidxs]
#             mfreq = freqs[freqs >= 10**(-2)][power[freqs >= 10**(-2)].argmax()] # ピーク周波数

#             dfs = pd.DataFrame([{'Datetime' : date_time,
#                                 'Peak_Freq' : mfreq,
#                                 'Peak_Cyc_T' : 1 / mfreq}])

#             df_freqs = df_freqs.append(dfs)

#         except:
#             with open('log.txt', 'a') as f:
#                 print(datetime.datetime.now(), '___error file : ', filelist[i], file=f)
#                 print(datetime.datetime.now(), '___error frame : ', j, file=f)
#                 continue

#     df_freq_all = df_freq_all.append(df_freqs)
#     df_freq_all.to_csv("../../../output/1_波浪/water_level/output_fft_all_data/fft_han_" +  filelist_)

        
# if __name__ == "__main__":
#     p = Pool(4)
#     p.map( calc_spectral, filelist )